In [1]:
%run ./utils.ipynb

----------

**Train:**
-------------

In [2]:
train_df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])

encoded_pixels_class_1 = []
encoded_pixels_class_2 = []
encoded_pixels_class_3 = []
encoded_pixels_class_4 = []
train_df1 = train_df[train_df['ClassId']=='1']
train_df2 = train_df[train_df['ClassId']=='2']
train_df3 = train_df[train_df['ClassId']=='3']
train_df4 = train_df[train_df['ClassId']=='4']
for image in set(train_df1['ImageId'].values.tolist()):
    encoded_pixels_class_1.append(train_df1[train_df1['ImageId']==image]['EncodedPixels'].values.tolist())
for image in set(train_df2['ImageId'].values.tolist()):
    encoded_pixels_class_2.append(train_df2[train_df2['ImageId']==image]['EncodedPixels'].values.tolist())
for image in set(train_df3['ImageId'].values.tolist()):
    encoded_pixels_class_3.append(train_df3[train_df3['ImageId']==image]['EncodedPixels'].values.tolist())
for image in set(train_df4['ImageId'].values.tolist()):
    encoded_pixels_class_4.append(train_df4[train_df4['ImageId']==image]['EncodedPixels'].values.tolist())
    
unique_images = list(set(train_df['ImageId'].values.tolist()))
data = {'ImageId': unique_images,
        'EP1': [item for sublist in encoded_pixels_class_1 for item in sublist],
        'EP2': [item for sublist in encoded_pixels_class_2 for item in sublist],
        'EP3': [item for sublist in encoded_pixels_class_3 for item in sublist],
        'EP4': [item for sublist in encoded_pixels_class_4 for item in sublist]}
train_df = pd.DataFrame(data)
print(train_df.shape)
train_df.head()

(12568, 5)


,ImageId,EP1,EP2,EP3,EP4
0,706efa6c9.jpg,NaN,NaN,NaN,NaN
1,c4260033a.jpg,NaN,NaN,NaN,NaN
2,94a5728a6.jpg,NaN,NaN,NaN,NaN
3,8f3044c4f.jpg,NaN,NaN,244994 95 245250 190 245506 189 245762 188 246...,NaN
4,f74f33985.jpg,NaN,NaN,46680 8 46928 18 47181 23 47434 27 47686 33 47...,NaN


In [3]:
train_df.to_csv('../output/train_df.csv', index = False)

In [14]:
def create_tiles(code, base, inference=False):
    new_ids = []
    new_EP1 = []
    new_EP2 = []
    new_EP3 = []
    new_EP4 = []
    
    # create image tiles
    path = f'{base}/{code}'
    tmp_code = code.replace('.jpg', '')
    img = cv2.imread(path)
    #img = img[:,32:-32]
    tiles = [img[x:x+256,y:y+400] for x in range(0,img.shape[0],256) for y in range(0,img.shape[1],400)]
    for i, tile in enumerate(tiles):
        tile = cv2.cvtColor(tile, cv2.COLOR_BGR2RGB)
        path = code.replace('.jpg', '')
        cv2.imwrite(f'{train_path}/{path}_{i}.png', tile)
        new_ids.append(f'{tmp_code}_{i}.png')
    
    if not inference:
        # create mask tiles
        image_df = train_df[train_df['ImageId'] == code]
        rles = [image_df['EP1'].values[0],image_df['EP2'].values[0],image_df['EP3'].values[0],image_df['EP4'].values[0]]
        masks = build_masks(rles, (256, 1600)).astype(np.uint8)
        #masks = masks[:,32:-32]
        tiles = [np.expand_dims(masks[x:x+256,y:y+400], axis=-1) for x in range(0,masks.shape[0],256) for y in range(0,masks.shape[1],400)]
        for i, tile in enumerate(tiles):
            tile = tile.flatten()
            tmp_rles = np.stack([np.reshape(np.where(tile==label, 1, 0), (256,400)) for label in range(5)], axis=2)
            tmp_rles = build_rles(tmp_rles)
            new_EP1.append(tmp_rles[0])
            new_EP2.append(tmp_rles[1])
            new_EP3.append(tmp_rles[2])
            new_EP4.append(tmp_rles[3])

        return new_ids, new_EP1, new_EP2, new_EP3, new_EP4
    
    else:
        return new_ids

def validate_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
train_path = '../output/class_train'
validate_path(train_path)

ids = []
EP1 = []
EP2 = []
EP3 = []
EP4 = []
for i, code in enumerate(tqdm(train_df['ImageId'])):
    new_ids, new_EP1, new_EP2, new_EP3, new_EP4 = create_tiles(
        code,
        base='../input/severstal-steel-defect-detection/train_images'
    )
    ids += new_ids
    EP1 += new_EP1
    EP2 += new_EP2 
    EP3 += new_EP3 
    EP4 += new_EP4

In [8]:
data = {'ImageId': ids, 'EP1': EP1, 'EP2': EP2, 'EP3': EP3, 'EP4': EP4}
train_df = pd.DataFrame(data)
print(len(train_df))
train_df.head()

NameError: name 'ids' is not defined

In [ ]:
train_df.to_csv('../output/train_df_crops.csv', index = False)

**Test:**
-------------

In [4]:
test_df = pd.read_csv('../input/severstal-steel-defect-detection/sample_submission.csv')
test_df['ImageId'] = test_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
test_df['ClassId'] = test_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])
print(test_df.shape)
test_df.head()

(7204, 4)


,ImageId_ClassId,EncodedPixels,ImageId,ClassId
0,004f40c73.jpg_1,1 1,004f40c73.jpg,1
1,004f40c73.jpg_2,1 1,004f40c73.jpg,2
2,004f40c73.jpg_3,1 1,004f40c73.jpg,3
3,004f40c73.jpg_4,1 1,004f40c73.jpg,4
4,006f39c41.jpg_1,1 1,006f39c41.jpg,1


In [5]:
test_df.to_csv('../output/test_df.csv', index = False)

In [16]:
train_path = '../output/class_test'
validate_path(train_path)

ids = []
for i, code in enumerate(tqdm(test_df['ImageId'])):
    new_ids = create_tiles(
        code,
        base='../input/severstal-steel-defect-detection/test_images',
        inference = True
    )
    ids += new_ids

In [17]:
data = {'ImageId': ids}
test_df = pd.DataFrame(data)
print(len(test_df))
test_df.head()

28816


,ImageId
0,004f40c73_0.png
1,004f40c73_1.png
2,004f40c73_2.png
3,004f40c73_3.png
4,004f40c73_0.png


In [18]:
test_df.to_csv('../output/test_df_crops.csv', index = False)